In [1]:
import os
import sys
sys.path.append('..')
LIB_PATH = os.getcwd() + "/art_library"
sys.path.append(LIB_PATH)
print(*sys.path, sep='\n')

/home/lukec/workspace/baard_exp2/experiments
/home/lukec/.vscode/extensions/ms-toolsai.jupyter-2020.12.414227025/pythonFiles
/home/lukec/.vscode/extensions/ms-toolsai.jupyter-2020.12.414227025/pythonFiles/lib/python
/usr/lib/python36.zip
/usr/lib/python3.6
/usr/lib/python3.6/lib-dynload

/home/lukec/workspace/baard_exp2/venv/lib/python3.6/site-packages
/home/lukec/workspace/baard_exp2/venv/lib/python3.6/site-packages/IPython/extensions
/home/lukec/.ipython
..
/home/lukec/workspace/baard_exp2/experiments/art_library


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import json

import matplotlib.pyplot as plt
import numpy as np
import torch
from tqdm import tqdm
import pandas as pd

In [4]:
from models.mnist import BaseModel
from models.cifar10 import  Resnet
from models.torch_util import predict_numpy
from defences.util import acc_on_adv
from defences.baard import ApplicabilityStage, ReliabilityStage, DecidabilityStage, BAARDOperator

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

device: cuda


In [6]:
dnn = BaseModel(use_prob=True).to(device)
resnet = Resnet(use_prob=True).to(device)

In [7]:
# f_dnn = os.path.join('..', 'results', 'mnist_200.pt')
f_resnet = os.path.join('..', 'results', 'cifar10_resnet_200.pt')

# dnn.load_state_dict(torch.load(f_dnn, map_location=device))
resnet.load_state_dict(torch.load(f_resnet, map_location=device))

<All keys matched successfully>

In [8]:
# Create BAARD
f_json_param = os.path.join('..', 'params', 'baard_cifar10_3.json')
with open(f_json_param) as j:
    baard_param = json.load(j)
print('Param:', baard_param)

n_classes = 10

Param: {'k_re': 10, 'k_de': 100, 'q1': 1.0, 'q2': 0.98, 'q3': 0.98, 'sequence': [True, True, True]}


In [9]:
stages_resnet = []
stages_resnet.append(ApplicabilityStage(n_classes=n_classes, quantile=baard_param['q1'], verbose=0))
stages_resnet.append(ReliabilityStage(n_classes=n_classes, k=baard_param['k_re'], quantile=baard_param['q2'], verbose=0))
stages_resnet.append(DecidabilityStage(n_classes=n_classes, k=baard_param['k_de'], quantile=baard_param['q3'], verbose=0))
detector_resnet = BAARDOperator(stages=stages_resnet)

In [10]:
# Load pretrained parameters
f_pre_data = os.path.join('..', 'result_0', 'cifar10_resnet_baard_s1_train_data.pt')
obj = torch.load(f_pre_data)
X_train_s1 = obj['X_s1']
X_train = obj['X']
y_train = obj['y']
print(X_train_s1.shape, X_train.shape, y_train.shape)

(48179, 3, 32, 32) (48179, 3, 32, 32) (48179,)


In [11]:
f_thresh_resnet = os.path.join('..', 'result_0', 'cifar10_resnet_baard_threshold.pt')
detector_resnet.fit(X=X_train, y=y_train, X_s1=X_train_s1)
detector_resnet.load(f_thresh_resnet)

Load from: ../result_0/cifar10_resnet_baard_threshold.pt


In [12]:
def get_data(model_name, att, e, d='adv'):
    f = 'cifar10_{}_{}_{}_{}.npy'.format(model_name, att, e, d)
    p = os.path.join('..', 'results', f)
    return np.load(p)

In [25]:
att = 'apgd'
e = 0.031
X = get_data('resnet', att, e, d='x')[:500]
adv = get_data('resnet', att, e)[:500]
y = get_data('resnet', att, e, d='y')[:500]
pred_adv = predict_numpy(resnet, adv, device)[:500]
print(np.mean(pred_adv == y))

0.008


In [19]:
s1 = detector_resnet.stages[0]
s2 = detector_resnet.stages[1]
s3 = detector_resnet.stages[2]

In [20]:
ks_s2 = np.arange(31, step=2)[1:]
acc_s2 = np.zeros_like(ks_s2, dtype=np.float)
fprs_s2 = np.zeros_like(ks_s2, dtype=np.float)

for i, k in enumerate(ks_s2):
    s2.k = k
    labelled_adv = s2.predict(adv, pred_adv)
    acc = acc_on_adv(pred_adv, y, labelled_adv)
    acc_s2[i] = acc

    labelled_x = s2.predict(X, y)
    fpr = np.mean(labelled_x)
    fprs_s2[i] = fpr

    print('[{:2d}] {:.4f} {:.4f}'.format(k, acc, fpr))

[ 2] 0.0380 0.0120
[ 4] 0.0400 0.0140
[ 6] 0.0420 0.0160
[ 8] 0.0460 0.0220
[10] 0.0520 0.0300
[12] 0.0580 0.0300
[14] 0.0600 0.0360
[16] 0.0620 0.0440
[18] 0.0640 0.0500
[20] 0.0660 0.0540
[22] 0.0680 0.0600
[24] 0.0760 0.0600
[26] 0.0760 0.0680
[28] 0.0840 0.0680
[30] 0.0880 0.0680


In [31]:
data = 'cifar10'
model_name = 'resnet'
data_s2_cifar10_resnet_fpr = {
    'k': ks_s2,
    'acc': acc_s2,
    'fpr': fprs_s2
}
df_s2 = pd.DataFrame(data_s2_cifar10_resnet_fpr)
print(df_s2)
file_name = '{}_{}_s2_r_{}_{}.csv'.format(data, model_name, att, int(e*1000))
print(file_name)
df_s2.to_csv(file_name, index=False)

     k    acc    fpr
0    2  0.038  0.012
1    4  0.040  0.014
2    6  0.042  0.016
3    8  0.046  0.022
4   10  0.052  0.030
5   12  0.058  0.030
6   14  0.060  0.036
7   16  0.062  0.044
8   18  0.064  0.050
9   20  0.066  0.054
10  22  0.068  0.060
11  24  0.076  0.060
12  26  0.076  0.068
13  28  0.084  0.068
14  30  0.088  0.068
cifar10_resnet_s2_r_apgd_31.csv


In [22]:
ks_s3 = np.arange(10, 200, step=10)
acc_s3 = np.zeros_like(ks_s3, dtype=np.float)
fprs_s3 = np.zeros_like(ks_s3, dtype=np.float)

for i, k in enumerate(ks_s3):
    s3.k = k
    labelled_adv = s2.predict(adv, pred_adv)
    acc = acc_on_adv(pred_adv, y, labelled_adv)
    acc_s3[i] = acc

    labelled_x = s3.predict(X, y)
    fpr = np.mean(labelled_x)
    fprs_s3[i] = fpr
    
    print('[{:3d}] {:.4f} {:.4f}'.format(k, acc, fpr))

[ 10] 0.0880 0.1660
[ 20] 0.0880 0.0820
[ 30] 0.0880 0.0420
[ 40] 0.0880 0.0320
[ 50] 0.0880 0.0240
[ 60] 0.0880 0.0260
[ 70] 0.0880 0.0240
[ 80] 0.0880 0.0200
[ 90] 0.0880 0.0180
[100] 0.0880 0.0140
[110] 0.0880 0.0180
[120] 0.0880 0.0140
[130] 0.0880 0.0140
[140] 0.0880 0.0140
[150] 0.0880 0.0160
[160] 0.0880 0.0180
[170] 0.0880 0.0180
[180] 0.0880 0.0160
[190] 0.0880 0.0140


In [32]:
data_s3_cifar10_resnet_fpr = {
    'k': ks_s3,
    'acc': acc_s3,
    'fpr': fprs_s3
}
df_s3 = pd.DataFrame(data_s3_cifar10_resnet_fpr)
print(df_s3)
file_name = '{}_{}_s3_r_{}_{}.csv'.format(data, model_name, att, int(e*1000))
print(file_name)
df_s3.to_csv(file_name, index=False)

      k    acc    fpr
0    10  0.088  0.166
1    20  0.088  0.082
2    30  0.088  0.042
3    40  0.088  0.032
4    50  0.088  0.024
5    60  0.088  0.026
6    70  0.088  0.024
7    80  0.088  0.020
8    90  0.088  0.018
9   100  0.088  0.014
10  110  0.088  0.018
11  120  0.088  0.014
12  130  0.088  0.014
13  140  0.088  0.014
14  150  0.088  0.016
15  160  0.088  0.018
16  170  0.088  0.018
17  180  0.088  0.016
18  190  0.088  0.014
cifar10_resnet_s3_r_apgd_31.csv
